In [9]:
from ase.build import make_supercell
from ase.calculators.eam import EAM
from ase.calculators.lammpsrun import LAMMPS
from ase.io import read

import catkit
from catkit import Gratoms
from catkit.gen.adsorption import get_adsorption_sites

from mcmc import mcmc_run, slab_energy, get_adsorption_coords
import os
import numpy as np

import sys
sys.path.append("/home/dux/")
from htvs.djangochem.pgmols.utils import surfaces

os.environ["PROJECT_DIR"] = os.getcwd()

In [10]:
# SiTiO3 from HTVS db
import sys
import os
import django
import numpy as np
from tqdm import tqdm
import pandas as pd
import pickle as pkl
sys.path.append('/home/dux/htvs/')
sys.path.append('/home/dux/htvs/djangochem/')

#from imports import *
#from django_imports import *
# from pymatgen.core.periodic_table import Element
# from chemconfigs.vasp.defaults import Magmom
# from ase.visualize import view

# setup the djhango settings file.  Change this to use the settings file that connects you to your desired database
os.environ["DJANGO_SETTINGS_MODULE"] = "djangochem.settings.orgel"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"  # CAUTION!!! LINE NECESSARY WHEN INTERACTING WITH DATABASE IN JUPYTER NOTEBOOK. DO NOT USE THIS LINE IF YOU ARE PLANNING ON SAVING/REMOVING THINGS FROM THE DATABASE!!!!!

# this must be run to setup access to the django settings and make database access work etc.
django.setup()
# from jobs.models import Job, JobConfig
# from django.contrib.contenttypes.models import ContentType
# from ase.io import write

from pgmols.models import Surface
# from analysis.structure.perovskite import PerovskiteAnalyzer
# from pymatgen.core.periodic_table import Element

unit_slab = Surface.objects.get(id=203512369).as_ase_gratoms()

# get supercell
multiplier = np.diag([4, 4, 1])

slab type <class 'catkit.gratoms.Gratoms'>


In [11]:
# use LAMMPS
# alloy_parameters = {
#     'pair_style': 'eam/alloy',
#     'pair_coeff': ['* * cu_ag_ymwu.eam.alloy Ag']
# }
# alloy_potential_file = os.path.join(os.environ["PROJECT_DIR"], 'cu_ag_ymwu.eam.alloy')
# alloy_calc = LAMMPS(files=[alloy_potential_file], keep_tmp_files=True, keep_alive=False, tmp_dir="/home/dux/surface_sampling/tmp_files")
# alloy_calc.set(**alloy_parameters)

# CaTiO3 from standard cell
# atoms = read('SrTiO3_mp-5229_conventional_standard.cif')
# slab, surface_atoms = surfaces.surface_from_bulk(atoms, [1,1,1], size=[4,4])
# slab.write('SrTiO3_111_4x4_pristine_slab.cif')

# CaTiO3 from Jackie's code saved as cif... can't get the sites
# atoms = read('SrTiO3_001_jackie_4x4.cif')
# slab = Gratoms(atoms)
# print("slab type", type(slab))

# slab.calc = alloy_calc
# print(f"Alloy calc pristine slab energy is {slab_energy(slab)}")

# slab.calc = pure_calc
# print(f"Pure calc pristine slab energy is {slab_energy(slab)}")

In [12]:
# manually get adsorption coords
coords, connectivity, sym_idx = get_adsorption_sites(slab, symmetry_reduced=False)

# get absolute adsorption coords
element = 'O'
ads = catkit.gratoms.Gratoms(element)

# ads_coords = get_adsorption_coords(slab, ads, connectivity)

/home/dux/miniconda3/envs/htvs/lib/python3.7/site-packages/catkit/gen/utils/coordinates.py:104: RuntimeWarning: invalid value encountered in sqrt
  a = np.sqrt(4 * d**2 * r[1]**2 - x0**2)
/home/dux/miniconda3/envs/htvs/lib/python3.7/site-packages/catkit/gen/utils/coordinates.py:122: RuntimeWarning: invalid value encountered in sqrt
  z = np.sqrt(r[0]**2 - x**2 - y**2)


In [13]:
slab = make_supercell(unit_slab, multiplier)

print("slab type", type(slab))
num_runs = 10
# no calc, testing to just get some states
history, energy_hist, frac_accept_hist, adsorption_count_hist = mcmc_run(num_runs=num_runs, temp=1, pot=0, alpha=1, slab=slab, calc=None, testing=True, element=element)

slab type <class 'catkit.gratoms.Gratoms'>
there are 229 atoms 
using slab calc None


In [1]:
import matplotlib.pyplot as plt
runs = range(1, num_runs+1)
# do the plots
fig, ax = plt.subplots(2, 2, figsize=(10, 8))
ax[0, 0].plot(runs, energy_hist)
ax[0, 0].set_xlabel("Iter")
ax[0, 0].set_ylabel("Energy (E)")
ax[0, 0].set_title("Energy (E) vs Sweeps")

ax[0, 1].plot(runs, frac_accept_hist)
ax[0, 1].set_xlabel("Iter")
ax[0, 1].set_ylabel("Fraction accepted")
ax[0, 1].set_title("Fraction accepted vs Sweeps")

ax[1, 1].plot(runs, np.array(list(adsorption_count_hist.values())).T)
ax[1, 1].set_xlabel("Iter")
ax[1, 1].set_ylabel("Adsorption count")
ax[1, 1].legend(adsorption_count_hist.keys())
ax[1, 1].set_title("Adsorption count vs Iterations")

fig.show()
fig.tight_layout()

NameError: name 'num_runs' is not defined